# ΕΠΕΞΕΡΓΑΣΙΑ ΦΥΣΙΚΗΣ ΓΛΩΣΣΑΣ - Εργασία 1
## Β. N-gram Language Models
**Μάθημα:**  Επεξεργασία Φυσικής Γλώσσας 

**Συγγραφέας:** Ιωάννης Kουτσούκης 

**Εκδόσεις:** 2025-03-22 | v.0.0.1

### 📁 Βήμα 1: Φόρτωση και Διαχωρισμός του Corpus

Χρησιμοποιούμε το corpus `treebank` από το NLTK, το οποίο περιέχει 199 αρχεία ειδήσεων από την Wall Street Journal. Διαχωρίζουμε τα δεδομένα σε:

- **170 αρχεία** για εκπαίδευση (training set).
- **29 αρχεία** για αξιολόγηση (test set).

Το corpus είναι ήδη χωρισμένο σε tokens και προτάσεις.


In [3]:
import nltk
from nltk.corpus import treebank
from collections import Counter

# 📌 Κατέβασμα corpus treebank αν δεν υπάρχει
nltk.download('treebank')

# 🔹 Λίστα με τα 199 αρχεία
files = treebank.fileids()

# 🔹 Διαχωρισμός σε train και test
train_files = files[:170]
test_files = files[170:]

# 🔹 Εξαγωγή προτάσεων
train_sents = [sent for file in train_files for sent in treebank.sents(file)]
test_sents = [sent for file in test_files for sent in treebank.sents(file)]

print(f"Πλήθος προτάσεων εκπαίδευσης: {len(train_sents)}")
print(f"Πλήθος προτάσεων αξιολόγησης: {len(test_sents)}")


[nltk_data] Downloading package treebank to
[nltk_data]     /Users/ioanniskoutsoukis/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


Πλήθος προτάσεων εκπαίδευσης: 3509
Πλήθος προτάσεων αξιολόγησης: 405


### Βήμα 2: Δημιουργία Λεξιλογίου και Αντικατάσταση με `<UNK>`

Υπολογίζουμε τις συχνότητες εμφάνισης όλων των tokens από το σύνολο εκπαίδευσης και κατασκευάζουμε το λεξιλόγιο **L**:

- Τα tokens που εμφανίζονται **λιγότερες από 3 φορές** αντικαθίστανται με `<UNK>`.
- Τα υπόλοιπα περιλαμβάνονται στο τελικό λεξιλόγιο **L**.


In [4]:
# 🔹 Συγκέντρωση όλων των tokens από τα κείμενα εκπαίδευσης
train_tokens = [token for sent in train_sents for token in sent]

# 🔹 Μέτρηση συχνοτήτων
freqs = Counter(train_tokens)

# 🔹 Κατασκευή λεξιλογίου: κρατάμε ό,τι έχει συχνότητα ≥ 3
vocab = {word for word, count in freqs.items() if count >= 3}

# 🔹 Συνάρτηση αντικατάστασης <UNK>
def replace_with_unk(sent, vocab):
    return [token if token in vocab else "<UNK>" for token in sent]

### Βήμα 3: Προετοιμασία Προτάσεων με `<BOS>` και `<EOS>`

Κάθε πρόταση περιβάλλεται από τα ειδικά tokens:

- `<BOS>` στην αρχή της πρότασης.
- `<EOS>` στο τέλος της πρότασης.

Αυτό μας επιτρέπει να δημιουργήσουμε ν-γράμματα που σέβονται τα όρια της πρότασης. Η αντικατάσταση με `<UNK>` γίνεται πριν την προσθήκη των ειδικών tokens.


In [5]:
def prepare_sentences(sents, vocab):
    result = []
    for sent in sents:
        replaced = replace_with_unk(sent, vocab)
        result.append(['<BOS>'] + replaced + ['<EOS>'])
    return result

# 🔹 Τελικές προτάσεις έτοιμες για δημιουργία n-grams
train_prepared = prepare_sentences(train_sents, vocab)
test_prepared = prepare_sentences(test_sents, vocab)


### Βήμα 4: Επιβεβαίωση αποτελεσμάτων βάσει Εκφώνησης


In [11]:
print(train_sents[0])
print(train_prepared[0])

print("Pierre εμφανίσεις:", freqs["Pierre"])
print("Vinken εμφανίσεις:", freqs["Vinken"])


['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.']
['<BOS>', '<UNK>', '<UNK>', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.', '<EOS>']
Pierre εμφανίσεις: 1
Vinken εμφανίσεις: 2


### Βήμα 5: Προετοιμασία για Συμπλήρωση Πίνακα (Ερώτημα 1)


In [15]:
from collections import Counter
import math
import re 

# Ορισμός βοηθητικής συνάρτησης για την εξαγωγή n-grams
def extract_ngrams(sentences, n):
    """
    Δέχεται λίστα προτάσεων (όπου κάθε πρόταση είναι λίστα από tokens)
    και επιστρέφει όλα τα n-grams που εξάγονται σε επίπεδο πρότασης.
    """
    ngrams = []
    for sent in sentences:
        ngrams.extend([tuple(sent[i:i+n]) for i in range(len(sent)-n+1)])
    return ngrams


# Lowercase μετατροπή
def to_lowercase(sents):
    return [[token.lower() for token in sent] for sent in sents]

# Abstract digits μετατροπή
def abstract_digits(sents):
    def abstract_token(token):
        return re.sub(r'\d', '#', token)
    return [[abstract_token(token) for token in sent] for sent in sents]


# Εκπαίδευση n-gram μοντέλου με add-k smoothing
class NgramModel:
    def __init__(self, n, k, train_data):
        """
        n: μέγεθος των n-grams (2 για bigram, 3 για trigram)
        k: παράμετρος εξομάλυνσης (add-k)
        train_data: λίστα προτάσεων με tokens (επεξεργασμένα με <UNK>, <BOS>, <EOS>)
        """
        self.n = n
        self.k = k
        self.train_data = train_data

        # Συλλογή n-grams και context (π.χ. bigram: (w1, w2), context: w1)
        self.ngrams = Counter(extract_ngrams(train_data, n))
        self.context = Counter(extract_ngrams(train_data, n - 1))

        # Λεξιλόγιο V (χρησιμοποιείται για smoothing)
        self.vocab = set(token for sent in train_data for token in sent)

    def prob(self, ngram):
        """
        Υπολογισμός πιθανοτήτων με add-k smoothing
        """
        prefix = ngram[:-1]
        return (self.ngrams[ngram] + self.k) / (self.context[prefix] + self.k * len(self.vocab))

    def perplexity(self, test_data):
        """
        Υπολογισμός perplexity
        """
        ngrams_test = extract_ngrams(test_data, self.n)
        N = len(ngrams_test)
        log_sum = 0
        for ngram in ngrams_test:
            p = self.prob(ngram)
            log_sum += math.log(p)
        return math.exp(-log_sum / N)

# Συνάρτηση αξιολόγησης για όλα τα μοντέλα (original, lowercase, digits)
def evaluate_all_versions(train_raw, test_raw):
    versions = {
        "Original text": (train_raw, test_raw),
        "Lowercase": (to_lowercase(train_raw), to_lowercase(test_raw)),
        "Abstract digits": (abstract_digits(train_raw), abstract_digits(test_raw))
    }

    settings = [(2, 1), (2, 0.01), (3, 1), (3, 0.01)]
    results = {v: {} for v in versions}

    for version, (train, test) in versions.items():
        for n, k in settings:
            model = NgramModel(n, k, train)
            ppl = model.perplexity(test)
            results[version][f'{n}-gram (k={k})'] = round(ppl, 2)

    return results

evaluate_all_versions(train_prepared, test_prepared)


{'Original text': {'2-gram (k=1)': 383.74,
  '2-gram (k=0.01)': 137.84,
  '3-gram (k=1)': 1505.81,
  '3-gram (k=0.01)': 464.06},
 'Lowercase': {'2-gram (k=1)': 349.46,
  '2-gram (k=0.01)': 130.43,
  '3-gram (k=1)': 1374.44,
  '3-gram (k=0.01)': 427.07},
 'Abstract digits': {'2-gram (k=1)': 336.26,
  '2-gram (k=0.01)': 122.12,
  '3-gram (k=1)': 1319.79,
  '3-gram (k=0.01)': 386.26}}

### Πίνακας & Συμπεράσματα ως προς το Perplexity

| Language model        | Original text | Lowercase   | Abstract digits |
|----------------------|---------------|-------------|-----------------|
| Bigrams (k = 1)      | 383.74        | 349.46      | 336.26          |
| Bigrams (k = 0.01)   | 137.84        | 130.43      | 122.12          |
| Trigrams (k = 1)     | 1505.81       | 1374.44     | 1319.79         |
| Trigrams (k = 0.01)  | 464.06        | 427.07      | 386.26          |
  
  
  
- **Πιο αποτελεσματικό μοντέλο**: Τα **bigrams** (2-grams) με **k = 0.01** είχαν τη **χαμηλότερη τιμή perplexity** σε όλες τις μορφές επεξεργασίας.
  
- **Τιμή του k**: Η τιμή **k = 0.01** είναι **σαφώς πιο κατάλληλη** από το k = 1, αφού μειώνει την εξομάλυνση και δίνει πιο ρεαλιστικές πιθανότητες σε σπάνια n-grams, βελτιώνοντας έτσι το perplexity.

- **Επίδραση lowercase**: Η μετατροπή όλων των χαρακτήρων σε πεζά (lowercase) **βελτίωσε τα αποτελέσματα** σε όλα τα μοντέλα. Αυτό είναι αναμενόμενο, καθώς ενοποιεί διαφορετικές μορφές της ίδιας λέξης (π.χ. `The` και `the`), μειώνοντας την πολυπλοκότητα του λεξιλογίου.

- **Επίδραση abstract digits**: Η αντικατάσταση αριθμών με αφηρημένους χαρακτήρες (`#`) **βελτίωσε περαιτέρω το perplexity**. Οι αριθμοί συχνά εμφανίζονται μία φορά (hapax legomena), οπότε η αφηρημένη αναπαράσταση μειώνει τη διάσταση του λεξιλογίου χωρίς απώλεια σημασιολογικής πληροφορίας.


### Βήμα 6: Παραγωγή Προτάσεων (Ερώτημα 2)


In [29]:
import random

def generate_sentence(model, max_length=30, verbose=False):
    """
    Δημιουργεί πρόταση με βάση το δοθέν μοντέλο n-gram.
    Επιλέγει τυχαία επόμενη λέξη ανάλογα με την πιθανότητα.
    - model: αντικείμενο NgramModel
    - max_length: μέγιστο μήκος πρότασης
    - verbose: αν είναι True, εμφανίζει τις πιθανότητες επιλογής
    """
    sentence = ['<BOS>']

    while len(sentence) < max_length:
        context = tuple(sentence[-(model.n - 1):]) if model.n > 1 else tuple()
        
        # Υποψήφια επόμενα tokens από λεξιλόγιο (εκτός <UNK>)
        candidates = [w for w in model.vocab if w != '<UNK>']
        
        # Υπολογισμός πιθανοτήτων για κάθε υποψήφιο
        probs = []
        for word in candidates:
            ngram = context + (word,)
            prob = model.prob(ngram)
            probs.append(prob)
        
        # Κανονικοποίηση πιθανοτήτων
        total = sum(probs)
        probs = [p / total for p in probs]

        # Τυχαία επιλογή λέξης με βάση τις πιθανότητες
        next_word = random.choices(candidates, weights=probs, k=1)[0]

        if verbose:
            print(f"Context: {context} -> Επιλογή: '{next_word}' (Prob={round(max(probs), 4)})")

        sentence.append(next_word)

        if next_word == '<EOS>':
            break

    return ' '.join(sentence[1:-1])  # αφαίρεση <BOS>, <EOS> από εμφάνιση

# 🔹 Συνδυασμοί που απαιτούνται
combinations = [
    (2, 1),     # Bigram, k=1
    (2, 0.01),  # Bigram, k=0.01
    (3, 1),     # Trigram, k=1
    (3, 0.01)   # Trigram, k=0.01
]

# 🔹 Παραγωγή προτάσεων
for n, k in combinations:
    print(f"\n Μοντέλο {n}-gram με k={k}:\n" + "-"*50)
    model = NgramModel(n, k, train_prepared)
    for i in range(1, 4):  # 3 προτάσεις για κάθε μοντέλο
        sentence = generate_sentence(model)
        print(f"* Πρόταση {i}: <BOS> {''.join(sentence)} <EOS>")



 Μοντέλο 2-gram με k=1:
--------------------------------------------------
* Πρόταση 1: <BOS> heart stopped medical Carla Carnival Old total start On globe Co plastic foreign number On Says Entertainment Sen. surprisingly Canada compound nearly Mercantile complex directly 51 equipment chemicals <EOS>
* Πρόταση 2: <BOS> 8.50 N.J eager where remove plunged employees national gives Two suffer gains seeing Several Markey basic roof-crush sense Price disciplinary acne providing century as looming 13 conference few <EOS>
* Πρόταση 3: <BOS> `` agreement widget people flat section 41 retired 30 So bridge covered multi-crystal turns In wait estimated household won standard expire Reupke becoming Money Co large front Computer <EOS>

 Μοντέλο 2-gram με k=0.01:
--------------------------------------------------
* Πρόταση 1: <BOS> In her , a share . <EOS>
* Πρόταση 2: <BOS> By earned fueled Trudeau refund card business appears US$ margins approach it to help *-1 always held media withdrawal year e

### Παρατηρήσεις & Υποθέσεις για την Ποιότητα των Παραγόμενων Προτάσεων

Κατά τη δημιουργία προτάσεων με χρήση των n-gram μοντέλων (Bigram/Trigram με τιμές k=1 και k=0.01), παρατηρήθηκε ότι:

- Οι παραγόμενες προτάσεις **δεν παρουσιάζουν συντακτική συνοχή** και **συχνά δεν βγάζουν νόημα** στη φυσική γλώσσα.
- Πολλές από αυτές μοιάζουν με **τυχαία ακολουθία λέξεων**, χωρίς λογική συνέχεια ή νοηματική σύνδεση.
- Σε αρκετές περιπτώσεις εμφανίζονται **ασυνήθιστοι συνδυασμοί**, π.χ. "`smoking activity smoking designers`", ή λέξεις που δεν θα συναντούσαμε εύκολα διαδοχικά σε φυσική χρήση.

#### 📌 Πιθανές αιτίες για τα παραπάνω φαινόμενα:

- Ίσως επειδή τα n-gram μοντέλα **λαμβάνουν υπόψη μόνο τα τοπικά συμφραζόμενα** (π.χ. μία ή δύο προηγούμενες λέξεις) και όχι το ευρύτερο νόημα της πρότασης.
- Επίσης, πιθανώς η **τυχαία επιλογή της αρχικής λέξης μετά το `<BOS>`** να επιτρέπει την εκκίνηση με λέξεις που δε συνάδουν με φυσική ροή πρότασης.
- Η τιμή **k=1** ενδεχομένως οδηγεί σε **υπερεξομάλυνση**, δηλαδή αποδυνάμωση της πληροφορίας που έχει το πραγματικό corpus, ενισχύοντας σπάνια n-grams.
- Αν και τα **trigram μοντέλα** βασίζονται σε μεγαλύτερα συμφραζόμενα, αυτό **δεν φαίνεται να επαρκεί** ώστε να δημιουργηθούν φυσικές προτάσεις, ενδεχομένως λόγω περιορισμένου μεγέθους ή ποικιλίας του training corpus.

Συνολικά, οι παραγόμενες προτάσεις παρέχουν ενδείξεις ότι τα στατιστικά n-gram μοντέλα, χωρίς βαθύτερη σημασιολογική κατανόηση, **δυσκολεύονται να παραγάγουν φυσικό και συνεκτικό λόγο**.
